### Import required packages

In [1]:
%matplotlib inline
import matplotlib.pyplot as g

from neuronunit.neuron.models import *
from neuronunit.tests import *
import neuronunit.neuroelectro
from quantities import nA, pA, s, ms, mV

from neuron import h

### Create a simple HH cell in NEURON

In [2]:
soma = h.Section()
soma.L = 100
soma.diam = 100
soma.cm = 1
soma.Ra = 180
soma.insert("pas")
soma.insert("hh")

### Create a NeuronUnit model using the cell

In [3]:
mod1 = SingleCellModel(hVar = h, \
                       section = soma, \
                       loc = 0.5, # Current and voltage injection and measurement location on the section \
                       name = "SimpleHHCellModel")

mod1.setIntegrationMethod("fixed")  # Or "variable"
mod1.setTimeStep(1/128.0 * ms) # 2^-n are preferable
#mod1.setTolerance() # Only needed if "variable"
mod1.setStopTime(2*s)

### Define tests to run

In [4]:
# Define which cell's properties to get
# 'nlx_anat_100201' is from Olfactory bulb mitral cell, 
# obtained from http://neuroelectro.org/neuron/129/ (Details)
neuron = {'nlex_id': 'nlx_anat_100201'} 

# Define injection current
i = {'injected_square_current': {'amplitude':10*nA, 'delay':1000*ms, 'duration':2000*ms } }

# Define the tests to perform on the cell
testTypes = [ \
    [InputResistanceTest, None],
    [RestingPotentialTest, None],
    [InjectedCurrentAPWidthTest, i], \
    [InjectedCurrentAPThresholdTest, i], \
    [InjectedCurrentAPAmplitudeTest, i],
]
     
tests = []

### Obtain Pooled Property Values from NeuroElectro.org

In [5]:
# No pooling example
testType = testTypes[0][0]
obs = testType.neuroelectro_summary_observation(neuron)
obs

Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?nlex=nlx_anat_100201&e__name=Input+Resistance


{'mean': array(130194117.647059) * ohm,
 'n': {u'id': 129,
  u'name': u'Olfactory bulb (main) mitral cell',
  u'neuron_db_id': 267,
  u'nlex_id': u'nlx_anat_100201'},
 'std': array(78794113.4751772) * ohm}

In [6]:
# Pooling example
testType = testTypes[0][0]
obs = testType.neuroelectro_pooled_observation(neuron, quiet=True)
obs

{'mean': array(87005386.514304) * ohm,
 'n': 864,
 'std': array(42626047.64067713) * ohm}

In [7]:
# Fetch NeuroElectro property values for each test
for t in xrange(len(testTypes)):
    testType = testTypes[t][0]
    params = testTypes[t][1]
    
    # Get the observations: property means and stds
    obs = testType.neuroelectro_pooled_observation(neuron)
    
    # Create a test instance using the observations
    test = testType(obs)
    
    if(params is not None):
        test.params = params
    
    tests.append(test)
    
# Create a test suite    
suite = sciunit.TestSuite("Mitral Cell Membrane Property Tests", tests)

### Evaluate the model

In [8]:
score = suite.judge(mod1)
score.view()

A Z score. A float indicating standardized difference from a reference mean.
The difference between the means of the observation and prediction divided by the standard deviation of the observation
A Z score. A float indicating standardized difference from a reference mean.
The difference between the means of the observation and prediction divided by the standard deviation of the observation
A Z score. A float indicating standardized difference from a reference mean.
The difference between the means of the observation and prediction divided by the standard deviation of the observation
A Z score. A float indicating standardized difference from a reference mean.
The difference between the means of the observation and prediction divided by the standard deviation of the observation
A Z score. A float indicating standardized difference from a reference mean.
The difference between the means of the observation and prediction divided by the standard deviation of the observation


/usr/local/lib/python2.7/site-packages/beautifulsoup4-4.4.1-py2.7.egg/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")



,Input resistance,Resting potential,Injected current AP width,Injected current AP threshold,Injected current AP amplitude
SimpleHHCellModel,Z = -2.00,Z = -3.51,Z = -0.59,Z = -4.70,Z = 15.64
